<a href="https://colab.research.google.com/github/Eitan177/Bioinformatics_Bootcamp/blob/main/informatics_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the GOAL Bioinformatics Bootcamp, 2024! To Setup this notebook please run the first three cells. This will download the packages you will need to participate

In [5]:
%%writefile assets/ngs_workshop.bed
chr17	7571720	7590868	TP53
chr3	178866311	178952152	PIK3CA
chr7	55086714	55324313	EGFR

Writing assets/ngs_workshop.bed


In [3]:
# NGS Bioinformatics Hands On Lab

# Install required packages
!apt-get update
!apt-get install -y fastp fastqc bwa samtools tabix
!pip install pysam

# Optional: Install additional tools if not available in Colab
!wget https://github.com/biod/sambamba/releases/download/v0.8.0/sambamba-0.8.0-linux-amd64-static.gz
!gunzip sambamba-0.8.0-linux-amd64-static.gz
!chmod +x sambamba-0.8.0-linux-amd64-static
!mv sambamba-0.8.0-linux-amd64-static /usr/local/bin/sambamba

# For ABRA2
!wget https://github.com/mozack/abra2/releases/download/v2.23/abra2-2.23.jar
!chmod +x abra2-2.23.jar

# For VarScan
!wget https://github.com/dkoboldt/varscan/raw/master/VarScan.v2.4.6.jar
!chmod +x VarScan.v2.4.6.jar

# for ANNOVAR
!wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
!tar -zxvf annovar.latest.tar.gz
!rm -r annovar.latest.tar.gz
!rm -r annovar/humandb


# Create directories for data and reference files
!mkdir -p data assets/gatk_b37 assets/humandb annovar

# Download pre-indexed reference genome
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.bwa_index.tar.gz -O assets/hg19_bwa_index.tar.gz



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,677 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,762 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692

In [ ]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.fa.gz -O assets/hg19fa_index.tar.gz

In [15]:
!tar -xzf assets/hg19_bwa_index.tar.gz -C assets/gatk_b37/

!mv assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.* assets/gatk_b37/
!mv assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa assets/gatk_b37/human_g1k_v37.fasta

mv: cannot stat 'assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.*': No such file or directory
mv: cannot stat 'assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set.fa': No such file or directory


In [17]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.fa.gz -O assets/hg19fa_index.gz
!gunzip assets/hg19fa_index.gz -C assets/gatk_b37/

--2025-03-20 20:41:53--  https://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/analysisSet/hg19.p13.plusMT.no_alt_analysis_set.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 863259336 (823M) [application/x-gzip]
Saving to: ‘assets/hg19fa_index.gz’

assets/hg19fa_index 100%[===================>] 823.27M  72.8MB/s    in 11s     

2025-03-20 20:42:05 (71.8 MB/s) - ‘assets/hg19fa_index.gz’ saved [863259336/863259336]

gzip: invalid option -- 'C'
Try `gzip --help' for more information.


In [18]:
# prompt: decompress /content/assets/hg19fa_index.gz and put in  assets/gatk_b37/


!gunzip /content/assets/hg19fa_index.gz -c > assets/gatk_b37/hg19.fa


In [14]:
# 1. Examine content of the FASTQ file
!zcat /content/ngs_ws_hd701_R1.fq.gz | head -n 20

# 2. Perform FASTQ processing prior to alignment
!fastp -i /content/ngs_ws_hd701_R1.fq.gz -o data/hd701_R1_processed.fq.gz -I /content/ngs_ws_hd701_R2.fq.gz -O data/hd701_R2_processed.fq.gz -z 4 -w 2 -h data/hd701_fq_qc.html

# 3. Generate FASTQ quality control data
!fastqc -o data -f fastq /content/ngs_ws_hd701_R1.fq.gz /content/ngs_ws_hd701_R2.fq.gz

# 4. Display QC report in Colab
from IPython.display import HTML
with open('data/hd701_fq_qc.html', 'r') as f:
    html_content = f.read()
HTML(html_content)







@NB501446:28:HMG5HBGX2:1:11101:10612:17818/1
GTCAATAATTCTGTTCTTAGGCCACTTTTCTCCCTTAATTTATTTCCTGTTCTTTCTCTTTTTAGCTCCCAGGGTT
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAAEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEE6EEEEEE
@NB501446:28:HMG5HBGX2:1:11101:16871:6164/1
GCCATTGCAGTGCAGAAGGGCAGTATCGGCAACCTCCTGGGCTTCCTAGAGGACAGCCTTGCCGCCACACAGTTCC
+
AAAAAEEEEEEEEEEAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE/EE/E<EEEEEEEEEEE/EEAAAEAEE
@NB501446:28:HMG5HBGX2:1:11101:17659:14137/1
GTTTGCTGGCCTGTTGAAAAATGACTGTAACAAAAGTGCTTCTGGTTATTTAACAGATGAAAATGAAGTGGGGTTT
+
AAAAAEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@NB501446:28:HMG5HBGX2:1:11101:23347:13640/1
ATATGTAAATGTAGTATAGGGAAGCTTCATAAGTCAGTCTCATCTGCAAATACTTGTGGGATTTTTAGCACAGCAA
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@NB501446:28:HMG5HBGX2:1:11101:7116:18158/1
CCGCCTGGAGAAGTTGTATAGCACTATGGTGCGCTTCCTCAGTGACCGAAAGAACCCGGTGTGCCGGGAGATGGCT
+
AAAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEE

In [19]:
# 5. Align sequence to GRCh37 - Update the BWA reference path
!bwa mem -M -v 1 -t 2 -R "@RG\\tID:hd701\\tSM:hd701\\tPL:ILLUMINA\\tPI:150\\tCN:lab" /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa data/hd701_R1_processed.fq.gz data/hd701_R2_processed.fq.gz > data/hd701_raw.sam



[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (144, 177, 224)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 384)
[M::mem_pestat] mean and std.dev: (186.75, 58.21)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 464)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -v 1 -t 2 -R @RG\tID:hd701\tSM:hd701\tPL:ILLUMINA\tPI:150\tCN:lab /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.p13.plusMT.no_alt_analysis_set.fa data/hd701_R1_processed.fq.gz data/hd701_R2_processed.fq.gz
[main] Real time: 33.805 sec; CPU: 8.244 sec


In [20]:
# 6. Convert the SAM file into a BAM file
!samtools view -Shu data/hd701_raw.sam > data/hd701_raw.bam

# 7. Sort and index raw BAM file
!sambamba sort -p -t 2 -o data/hd701_sorted.bam data/hd701_raw.bam

# 8. Mark PCR duplicates in BAM file
!sambamba markdup -t 2 -p data/hd701_sorted.bam data/hd701_dedup.bam


sambamba 0.8.0
 by Artem Tarasov and Pjotr Prins (C) 2012-2020
    LDC 1.10.0 / DMD v2.080.1 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.4)

Writing sorted chunks to temporary directory...
[==============================================================================]

sambamba 0.8.0
 by Artem Tarasov and Pjotr Prins (C) 2012-2020
    LDC 1.10.0 / DMD v2.080.1 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.4)

finding positions of the duplicate reads in the file...
[==============================================================================]
  sorted 4959 end pairs
     and 0 single ends (among them 0 unmatched pairs)
  collecting indices of duplicate reads...   done in 2 ms
  found 860 duplicates
collected list of positions in 0 min 0 sec
marking duplicates...
[==============================================================================]
collected list of positions in 0 min 0 sec


In [ ]:
# 9. Perform indel realignment - Update the reference path
!java -jar abra2-2.23.jar --in data/hd701_dedup.bam --out data/hd701_realigned.bam --ref /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa --threads 2 --targets assets/ngs_ws.bed --index --tmpdir /tmp/ > data/hd701_abra.log

# 10. Inspect aligned sequences (show first few lines)
!samtools view -h data/hd701_realigned.bam | head -n 20



In [27]:
!samtools index data/hd701_realigned.bam

In [28]:
!ls -l /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa
!head -n 5 /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa

-rw-r--r-- 1 root root 3146301083 Mar 20 20:43 /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa
>chr1  AC:CM000663.1  gi:224384768  LN:249250621  rl:Chromosome  M5:d949d5ece55a7f11abb62abfe947815d  AS:GRCh37.p13
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [29]:
!ls -l assets/ngs_ws.bed
!head -n 5 assets/ngs_ws.bed

-rw-r--r-- 1 root root 186 Mar 20 20:18 assets/ngs_ws.bed
1	27105337	27107952	ARID1A
3	178951769	178952287	PIK3CA_ex21
7	55241584	55241766	EGFR_ex18
7	140453045	140453223	BRAF_ex15
12	25398208	25398329	KRAS_ex2


In [33]:
!java -version
!ls -l VarScan.v2.4.6.jar

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
-rwxr-xr-x 1 root root 126611 Mar 20 19:59 VarScan.v2.4.6.jar


In [37]:
!head -n 10 data/hd701_mpileup.txt
!wc -l data/hd701_mpileup.txt

chr1	27105338	T	8	........	EEEEEEEE
chr1	27105339	G	8	........	EEEEAEEE
chr1	27105340	T	9	.$.......^].	EEEEEEEEA
chr1	27105341	T	7	.......	EEEEEEA
chr1	27105342	G	7	.......	EAEEEEA
chr1	27105343	G	8	........	EEEEEEEA
chr1	27105344	G	8	........	EEEEEEEA
chr1	27105345	C	8	........	EEEAEEEE
chr1	27105346	A	8	........	AEEEEEEE
chr1	27105347	T	8	........	EEEEEEEE
8078 data/hd701_mpileup.txt


In [36]:
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_ws.bed -f /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa data/hd701_realigned.bam > data/hd701_mpileup.txt
!wc -l data/hd701_mpileup.txt
# Step 2: Run VarScan on the mpileup file
#!cat data/hd701_mpileup.txt | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/hd701_raw.vcf

[mpileup] 1 samples in 1 input files
8078 data/hd701_mpileup.txt


In [38]:
!# 11. Call variant - Update the reference path
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_ws.bed -f /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa data/hd701_realigned.bam | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/hd701_raw.vcf

[mpileup] 1 samples in 1 input files
Only variants will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.05
Min avg qual:	30
P-value thresh:	0.05
Input stream not ready, waiting for 5 seconds...
Reading input from STDIN
8078 bases in pileup file
14 variant positions (10 SNP, 4 indel)
0 were failed by the strand-filter
14 variant positions reported (10 SNP, 4 indel)


In [39]:
# 12. Compress and index VCF file
!bgzip data/hd701_raw.vcf
!tabix -p vcf data/hd701_raw.vcf.gz

# 13. Inspect variants in a raw VCF file
!zcat data/hd701_raw.vcf.gz | head -n 20



[bgzip] data/hd701_raw.vcf.gz already exists; do you wish to overwrite (y or n)? y
##fileformat=VCFv4.3
##source=VarScan2
##INFO=<ID=ADP,Number=1,Type=Integer,Description="Average per-sample depth of bases with Phred score >= 30">
##INFO=<ID=WT,Number=1,Type=Integer,Description="Number of samples called reference (wild-type)">
##INFO=<ID=HET,Number=1,Type=Integer,Description="Number of samples called heterozygous-variant">
##INFO=<ID=HOM,Number=1,Type=Integer,Description="Number of samples called homozygous-variant">
##INFO=<ID=NC,Number=1,Type=Integer,Description="Number of samples not called">
##FILTER=<ID=str10,Description="Less than 10% or more than 90% of variant supporting reads on one strand">
##FILTER=<ID=indelError,Description="Likely artifact due to indel reads at this position">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=SDP,Number=1,Type=Integer,Description="Raw Read Depth a

In [41]:
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar refGene assets/humandb/
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar cytoBand assets/humandb/
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar clinvar_20150330 assets/humandb/
!perl annovar/annotate_variation.pl -buildver hg19 -downdb -webfrom annovar cosmic85 assets/humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg19 build version, with files saved at the 'assets/humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_cytoBand.txt.gz ... Failed
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_cytoBand.txt.idx.gz ... Failed
NOTICE: Web-based checking to see whether ANNOVAR new vers

In [64]:
!perl /content/annovar/annotate_variation.pl -buildver hg19 -downdb cytoBand assets/humandb/


NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/cytoBand.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg19 build version, with files saved at the 'assets/humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/hg19_cosmic85.txt.txt.gz ... Failed


In [67]:
!perl /content/annovar/annotate_variation.pl -buildver hg19 -downdb hg19_cosmic assets/humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/hg19_cosmic.txt.gz ... Failed


In [ ]:
annotate_variation.pl -filter -build hg19 -out ex4 -dbtype cosmic68 ex4.avinput humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_cytoBand.txt.gz ... Failed
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg19_cytoBand.txt.idx.gz ... Failed


In [62]:
# 14. Annotate variants (requires ANNOVAR to be set up)
!perl annovar/table_annovar.pl data/hd701_raw.vcf.gz assets/humandb/ -out data/hd701_annotated -buildver hg19 -remove -nastring . -otherinfo -vcfinput -thread 2 -maxgenethread 2 -protocol refGene,cytoBand,cosmic85,clinvar_20150330 -operation g,r,f,f

# 15. Inspect annotated VCF file
!head -n 20 data/hd701_annotated.hg19_multianno.vcf

NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
Error: the required database file assets/humandb/hg19_cosmic85.txt does not exist.
head: cannot open 'data/hd701_annotated.hg19_multianno.vcf' for reading: No such file or directory


In [24]:
# 11. Call variant - Update the reference path
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_ws.bed -f /content/assets/gatk_b37/hg19.p13.plusMT.no_alt_analysis_set/hg19.fa data/hd701_realigned.bam | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/hd701_raw.vcf




# 16. Visualize BAM files with IGV.js
!pip install igv-jupyter
import igv
viewer = igv.Browser({
    "genome": "hg19",
    "locus": "chr17:7,571,720-7,590,868"  # TP53 region
})
viewer.load_track({
    "name": "HD701 Alignment",
    "url": "data/hd701_realigned.bam",
    "indexURL": "data/hd701_realigned.bam.bai",
    "format": "bam"
})
viewer

# Download results to your local machine
from google.colab import files
!zip -r hd701_results.zip data/
files.download('hd701_results.zip')

Only variants will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.05
Min avg qual:	30
P-value thresh:	0.05
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
[mpileup] 1 samples in 1 input files
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
Input stream not ready, waiting for 5 seconds...
[bgzip] data/hd701_raw.vcf.gz already exists; do you wish to 

AttributeError: module 'igv' has no attribute 'Browser'

In [7]:
!fastqc -o data -f fastq /content/ngs_ws_hd701_R1.fq.gz /content/ngs_ws_hd701_R2.fq.gz

Started analysis of ngs_ws_hd701_R1.fq.gz
Approx 20% complete for ngs_ws_hd701_R1.fq.gz
Approx 40% complete for ngs_ws_hd701_R1.fq.gz
Approx 60% complete for ngs_ws_hd701_R1.fq.gz
Approx 80% complete for ngs_ws_hd701_R1.fq.gz
Approx 100% complete for ngs_ws_hd701_R1.fq.gz
Analysis complete for ngs_ws_hd701_R1.fq.gz
Started analysis of ngs_ws_hd701_R2.fq.gz
Approx 20% complete for ngs_ws_hd701_R2.fq.gz
Approx 40% complete for ngs_ws_hd701_R2.fq.gz
Approx 60% complete for ngs_ws_hd701_R2.fq.gz
Approx 80% complete for ngs_ws_hd701_R2.fq.gz
Approx 100% complete for ngs_ws_hd701_R2.fq.gz
Analysis complete for ngs_ws_hd701_R2.fq.gz


In [4]:
import os
import subprocess

# Define input and output paths
fastq_r1 = '/content/ngs_ws_hd701_R1.fq.gz'
fastq_r2 = '/content/ngs_ws_hd701_R2.fq.gz'
ref_genome = 'assets/gatk_b37/human_g1k_v37.fasta'
output_prefix = 'data/ngs_ws_hd701'
threads = 4  # Adjust based on your environment

# Step 1: Quality control with FastQC (optional but recommended)
print("Running FastQC...")
os.makedirs('data/fastqc', exist_ok=True)
subprocess.run(['fastqc', fastq_r1, fastq_r2, '-o', 'data/fastqc'])

# Step 2: Quality trimming with fastp
print("Running fastp for quality trimming...")
subprocess.run([
    'fastp',
    '-i', fastq_r1,
    '-I', fastq_r2,
    '-o', f'{output_prefix}_trimmed_R1.fastq.gz',
    '-O', f'{output_prefix}_trimmed_R2.fastq.gz',
    '--json', f'{output_prefix}_fastp.json',
    '--html', f'{output_prefix}_fastp.html'
])

# Step 3: Align to reference with BWA-MEM
print("Aligning to reference genome...")
sam_file = f'{output_prefix}.sam'
bwa_cmd = [
    'bwa', 'mem',
    '-t', str(threads),
    '-R', f'@RG\\tID:ngs_ws_hd701\\tSM:ngs_ws_hd701\\tPL:ILLUMINA',
    ref_genome,
    f'{output_prefix}_trimmed_R1.fastq.gz',
    f'{output_prefix}_trimmed_R2.fastq.gz'
]

with open(sam_file, 'w') as f:
    subprocess.run(bwa_cmd, stdout=f)

# Step 4: Convert SAM to BAM
print("Converting SAM to BAM...")
bam_file = f'{output_prefix}.bam'
subprocess.run(['samtools', 'view', '-b', '-h', '-o', bam_file, sam_file])

# Step 5: Sort the BAM file
print("Sorting BAM...")
sorted_bam = f'{output_prefix}.sorted.bam'
subprocess.run(['samtools', 'sort', '-o', sorted_bam, bam_file])

# Step 6: Index the BAM file
print("Indexing BAM...")
subprocess.run(['samtools', 'index', sorted_bam])

# Step 7: Mark duplicates with sambamba
print("Marking duplicates...")
dedup_bam = f'{output_prefix}.dedup.bam'
subprocess.run([
    'sambamba', 'markdup',
    '--tmpdir=/tmp',
    '-t', str(threads),
    sorted_bam,
    dedup_bam
])

# Step 8: Generate alignment statistics
print("Generating alignment statistics...")
subprocess.run(['samtools', 'flagstat', dedup_bam, '-o', f'{output_prefix}.flagstat.txt'])

# Cleanup intermediate files (optional)
print("Cleaning up intermediate files...")
os.remove(sam_file)
os.remove(bam_file)

print("Pipeline completed successfully!")
print(f"Final BAM file: {dedup_bam}")

Running FastQC...
Running fastp for quality trimming...
Aligning to reference genome...
Converting SAM to BAM...
Sorting BAM...
Indexing BAM...
Marking duplicates...
Generating alignment statistics...
Cleaning up intermediate files...


FileNotFoundError: [Errno 2] No such file or directory: 'data/ngs_ws_hd701.bam'

In [2]:
!wget -O data/NA12878_chr17_R1.fastq.gz https://github.com/genome-in-a-bottle/giab_data_indexes/raw/master/NA12878/sequence/NIST7035_TAAGGCGA_L001_R1_001_trimmed_example.fastq.gz
!wget -O data/NA12878_chr17_R2.fastq.gz https://github.com/genome-in-a-bottle/giab_data_indexes/raw/master/NA12878/sequence/NIST7035_TAAGGCGA_L001_R2_001_trimmed_example.fastq.gz

# If the above links don't work, try these alternative small datasets
# !wget -O data/NA12878_chr17_R1.fastq.gz https://zenodo.org/record/1251112/files/NA12878_chr17_mini_R1.fastq.gz
# !wget -O data/NA12878_chr17_R2.fastq.gz https://zenodo.org/record/1251112/files/NA12878_chr17_mini_R2.fastq.gz

# 1. Examine content of the FASTQ file
!zcat data/NA12878_chr17_R1.fastq.gz | head -n 20

# 2. Perform FASTQ processing prior to alignment
!fastp -i data/NA12878_chr17_R1.fastq.gz -o data/NA12878_chr17_R1_processed.fq.gz -I data/NA12878_chr17_R2.fastq.gz -O data/NA12878_chr17_R2_processed.fq.gz -z 4 -w 2 -h data/NA12878_fq_qc.html

# 3. Generate FASTQ quality control data
!fastqc -o data -f fastq data/*.gz

# 4. Display QC report in Colab
from IPython.display import HTML
with open('data/NA12878_fq_qc.html', 'r') as f:
    html_content = f.read()
HTML(html_content)

# 5. Align sequence to GRCh37
!bwa mem -M -v 1 -t 2 -R "@RG\\tID:NA12878\\tSM:NA12878\\tPL:ILLUMINA\\tPI:150\\tCN:lab" assets/gatk_b37/human_g1k_v37.fasta data/NA12878_chr17_R1_processed.fq.gz data/NA12878_chr17_R2_processed.fq.gz > data/NA12878_raw.sam

# 6. Convert the SAM file into a BAM file
!samtools view -Shu data/NA12878_raw.sam > data/NA12878_raw.bam

# 7. Sort and index raw BAM file
!sambamba sort -p -t 2 -o data/NA12878_sorted.bam data/NA12878_raw.bam

# 8. Mark PCR duplicates in BAM file
!sambamba markdup -t 2 -p data/NA12878_sorted.bam data/NA12878_dedup.bam

# 9. Perform indel realignment
!java -jar abra2-2.23.jar --in data/NA12878_dedup.bam --out data/NA12878_realigned.bam --ref assets/gatk_b37/human_g1k_v37.fasta --threads 2 --targets assets/ngs_workshop.bed --index --tmpdir /tmp/ > data/NA12878_abra.log

# 10. Inspect aligned sequences (show first few lines)
!samtools view -h data/NA12878_realigned.bam | head -n 20

# 11. Call variant
!samtools mpileup -BA -q 20 -Q 30 -d 4000 -l assets/ngs_workshop.bed -f assets/gatk_b37/human_g1k_v37.fasta data/NA12878_realigned.bam | java -Xmx4G -jar VarScan.v2.4.6.jar mpileup2vcf --min-coverage 8 --min-var-freq 0.05 --p-value 0.05 --min-avg-qual 30 --strand-filter 1 --output-vcf 1 --variants > data/NA12878_raw.vcf

# 12. Compress and index VCF file
!bgzip data/NA12878_raw.vcf
!tabix -p vcf data/NA12878_raw.vcf.gz

# 13. Inspect variants in a raw VCF file
!zcat data/NA12878_raw.vcf.gz | head -n 20

# 14. Annotate variants (requires ANNOVAR to be set up)
!perl annovar/table_annovar.pl data/NA12878_raw.vcf.gz assets/humandb/ -out data/NA12878_annotated -buildver hg19 -remove -nastring . -otherinfo -vcfinput -thread 2 -maxgenethread 2 -protocol refGene,cytoBand,cosmic85,clinvar_20150330 -operation g,r,f,f

# 15. Inspect annotated VCF file
!head -n 20 data/NA12878_annotated.hg19_multianno.vcf

# 16. Visualize BAM files with IGV.js
!pip install igv-jupyter
import igv
viewer = igv.Browser({
    "genome": "hg19",
    "locus": "chr17:7,571,720-7,590,868"  # TP53 region
})
viewer.load_track({
    "name": "NA12878 Alignment",
    "url": "data/NA12878_realigned.bam",
    "indexURL": "data/NA12878_realigned.bam.bai",
    "format": "bam"
})
viewer

# Download results to your local machine
from google.colab import files
!zip -r ngs_results.zip data/
files.download('ngs_results.zip')

--2025-03-15 23:27:32--  https://github.com/genome-in-a-bottle/giab_data_indexes/raw/master/NA12878/sequence/NIST7035_TAAGGCGA_L001_R1_001_trimmed_example.fastq.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-15 23:27:33 ERROR 404: Not Found.

--2025-03-15 23:27:33--  https://github.com/genome-in-a-bottle/giab_data_indexes/raw/master/NA12878/sequence/NIST7035_TAAGGCGA_L001_R2_001_trimmed_example.fastq.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-15 23:27:33 ERROR 404: Not Found.


gzip: data/NA12878_chr17_R1.fastq.gz: unexpected end of file
ERROR: igzip: Error invalid gzip header found: data/NA12878_chr17_R1.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
Failed to process data/NA12878

FileNotFoundError: [Errno 2] No such file or directory: 'data/NA12878_fq_qc.html'